<a href="https://colab.research.google.com/github/zakariafaouzi/LLM-Models/blob/main/Darija_MSA_AraBART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Mar 14 18:18:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!git clone https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install datasets
!pip install evaluate
!pip install sentencepiece
!pip install tensorboardX
!pip install accelerate -U
!pip install sacrebleu

Cloning into 'transformers'...
remote: Enumerating objects: 191407, done.
remote: Counting objects: 100% (22107/22107), done.
remote: Compressing objects: 100% (1941/1941), done.
remote: Total 191407 (delta 21502), reused 20230 (delta 20150), pack-reused 169300
Receiving objects: 100% (191407/191407), 199.60 MiB | 21.24 MiB/s, done.
Resolving deltas: 100% (136379/136379), done.
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-hi28pr4l
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-hi28pr4l
  Resolved https://github.com/huggingface/transformers.git to commit 23db187d9223cfbd535a3a76fb518ca2c1429633
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.39.0.dev0-py3-none-any.whl size=8681280 sha256=bad29e8b2eea581faa9d6ad25dded11ae95eb90c2cfb814ce

In [ ]:
import urllib

urllib.request.urlretrieve("https://onedrive.live.com/download?resid=758EF4B4D89BDE45%214789&authkey=!ABCcTMoUFdERYYI", 'test_4000.json')
urllib.request.urlretrieve("https://onedrive.live.com/download?resid=758EF4B4D89BDE45%214790&authkey=!ALwWFPfNjxM737o", 'train_6000.json')

('train_6000.json', <http.client.HTTPMessage at 0x7d7ef2b3e950>)

In [ ]:
import json

path = "train_6000.json"

# Ouvrir le fichier et lire les 100 premiers caractères
with open(path, 'r') as file:
    content = file.read(100)
    print(content)

# Essayez de charger le contenu comme JSON
try:
    json_data = json.loads(content)
    print("Le fichier est un JSON valide.")
except json.JSONDecodeError as e:
    print(f"Erreur JSON : {e}")


{"translation": {"idx": 4000, "English": "Good evening!", "Darija": "ماساء لخاير!", "MSA": "مساء الخ
Erreur JSON : Unterminated string starting at: line 1 column 92 (char 91)


## Preprocessing part

In [ ]:
import pandas as pd

In [ ]:
our_data = pd.read_excel("group_b.xlsx")
our_data = our_data[["idx", "English", "Darija", "MSA", "Checkbox", "Darija corrected"]]
our_data.head(3)

,idx,English,Darija,MSA,Checkbox,Darija corrected
0,4000,Good evening!,ماساء لخاير!,مساء الخير!,True,مْسا الخير!
1,4001,What do you have on the pressure?,أش عانداك علا ضاغط?,ماذا لديك عن الضغط؟,True,آش عندك على الضغط ؟
2,4002,On the pressure?,علا ضاغط?,تحت الضغط؟,True,على الضغط؟


In [ ]:
our_data['Darija'] = our_data.apply(lambda row: row['Darija corrected'] if pd.notnull(row['Darija corrected']) else row['Darija'], axis=1)


In [ ]:
our_data = our_data[our_data['Checkbox'] == True]
our_data = our_data[["idx", "English", "Darija", "MSA"]]
our_data.head()

,idx,English,Darija,MSA
0,4000,Good evening!,مْسا الخير!,مساء الخير!
1,4001,What do you have on the pressure?,آش عندك على الضغط ؟,ماذا لديك عن الضغط؟
2,4002,On the pressure?,على الضغط؟,تحت الضغط؟
3,4003,i don't follow.,مافهمتش.,لا أفهم.
4,4004,I am pretty stressed out with work at the moment.,أنا متوتر بزاف مع الخدمة هاد الساعة.,أنا متوتر جدًا بسبب العمل في الوقت الحالي.


In [ ]:
data2 = pd.read_excel("translated_phrases.xlsx")
data2.head(2)

,idx,English,Darija,MSA
0,11981,We need to save our energy for the climb and t...,غادي يخصنا نحتافضو علا طاقا ديالنا علا قبل طلع...,نحن بحاجة إلى توفير طاقتنا للتسلق وطول المشي إ...
1,11982,Absolutely.,ب طابيعت لحال.,تماماً تماماً


In [ ]:
our_data = pd.concat([our_data,data2])
our_data

,idx,English,Darija,MSA
0,4000,Good evening!,مْسا الخير!,مساء الخير!
1,4001,What do you have on the pressure?,آش عندك على الضغط ؟,ماذا لديك عن الضغط؟
2,4002,On the pressure?,على الضغط؟,تحت الضغط؟
3,4003,i don't follow.,مافهمتش.,لا أفهم.
4,4004,I am pretty stressed out with work at the moment.,أنا متوتر بزاف مع الخدمة هاد الساعة.,أنا متوتر جدًا بسبب العمل في الوقت الحالي.
...,...,...,...,...
3976,15957,We see technology as a tool for empowerment an...,احنا نشوف التكنولوجيا كأداة للتمكين والتغيير ا...,نحن نرى التكنولوجيا كأداة للتمكين والتغيير الا...
3977,15958,The Internet has transformed how we access inf...,الإنترنت غير كيفية الوصول للمعلومات والتواصل م...,الإنترنت غير كيفية الوصول للمعلومات والتواصل م...
3978,15959,She enjoys building mobile apps that solve rea...,هي تستمتع ببناء تطبيقات الهواتف المحمولة التي ...,هي تستمتع ببناء تطبيقات الهواتف المحمولة التي ...
3979,15960,He is interested in the intersection of techno...,هو مهتم بتقاطع التكنولوجيا والرعاية الصحية، ال...,هو مهتم بتقاطع التكنولوجيا والرعاية الصحية، ال...


### Spliter la donnée et la mettre sous format json

In [ ]:
import json

In [ ]:


def generate_jsonlines(dataframe, output_file):
    with open(output_file, 'w', encoding='utf-8') as file:
        for _, row in dataframe.iterrows():
            entry = {
                "translation": {
                    "idx": row["idx"],
                    "English": row["English"],
                    "Darija": row["Darija"],
                    "MSA": row["MSA"]
                }
            }
            json_line = json.dumps(entry, ensure_ascii=False)
            file.write(json_line + '\n')

# Generate JSONLINES files for training and testing
generate_jsonlines(our_data, 'train_jsonl.json')


In [ ]:
import json

def txt_to_json(input_file, output_file):
    with open(input_file, 'r') as txt_file:
        txt_content = txt_file.readlines()

    data = {txt_content}

    with open(output_file, 'w') as json_file:
        json.dump(data, json_file, indent=4)

input_file = 'train_finale.txt'
output_file = 'train_finale.json'

# Appeler la fonction pour convertir le fichier texte en JSON
txt_to_json(input_file, output_file)


In [ ]:
!(python3 transformers/examples/pytorch/translation/run_translation.py \
    --model_name_or_path moussaKam/arabart \
    --do_train \
    --do_eval \
    --source_lang Darija \
    --target_lang MSA \
    --train_file ./train_finale.json \
    --validation_file ./test_4000.json \
    --output_dir ./tmp/tst-translation \
    --per_device_train_batch_size=16 \
    --per_device_eval_batch_size=16 \
    --save_steps=200 --eval_steps=200 \
    --logging_steps=200 \
    --save_total_limit=15 \
    --overwrite_output_dir \
    --push_to_hub=False \
    --preprocessing_num_workers=8 \
    --dataloader_num_workers=2 \
    --dataloader_prefetch_factor=2 \
    --num_train_epochs=5 \
    --evaluation_strategy='steps' \
    --save_strategy='steps' \
    --load_best_model_at_end=True \
    --predict_with_generate)

2024-03-14 18:22:35.594006: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-14 18:22:35.594112: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-14 18:22:35.700189: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-14 18:22:36.975228: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
03/14/2024 18:22:40 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
03/14/2024 18:22:40 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gp

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_path = 'tmp/tst-translation'
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained('moussaKam/arabart')

seq = "غادي نقسمو الحساب؟"
tok = tokenizer.encode(seq, return_tensors='pt')

res = model.generate(tok)
tokenizer.decode(res[0]) # gramatically correct but wrong translation

'</s><s> هل يمكننا تقسيم الفاتورة؟</s>'

In [ ]:
print(tokenizer.decode(res[0]))

</s><s> ولكن لا يوجد أي تعليق مخلوط في هذا</s>
